<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Qonto - Relevé de compte augmenté
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Qonto/Qonto_Get_statement_detailed.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

## Input

### Import des librairies

In [ ]:
from naas_drivers import qonto, emailbuilder
from datetime import datetime, timedelta
import pandas as pd
import naas

### Configuration utilisateur

#### Qonto credentials

In [ ]:
QONTO_USER_ID = 'YOUR_USER_ID'
QONTO_API_KEY = 'YOUR_API_KEY'

#### Email information

In [ ]:
# List of emails
EMAIL_TO = ["your_email_adresse"]

# Subject diplay in email
EMAIL_SUBJECT = f"🏛️ Qonto - Votre relevé de compte augmenté du {datetime.now().strftime('%d/%m/%Y')}"

#### Periode d'analyse

In [ ]:
# Date from in format YYYY-MM-DD
DATE_FROM = "2021-01-01"

# Date to in format YYYY-MM-DD (default = TODAY)
DATE_TO = datetime.now().strftime("%Y-%m-%d")

# Number of days to get last transactions, must be negative integer
LAST_TRANSACTIONS = -7

#### Nom des fichiers partagés dans l'email

In [ ]:
GRAPH_FILE = "graph_account_statement.html"
GRAPH_IMG = "graph_account_statement.jpeg"
TABLE_FILE = "account_statement.xlsx"

### Récupération du relevé consolidé

In [ ]:
# Colonne to consolidate (DATE already included), if empty return only DATE, AMOUNT, POSITION
to_conso = ["TRANSACTION_ID",
            "LABEL",
            "OPERATION_TYPE"]

df_statement = qonto.connect(QONTO_USER_ID, QONTO_API_KEY).statement.consolidated(to_conso=to_conso,
                                                                                  date_from=DATE_FROM,
                                                                                  date_to=DATE_TO)
df_statement

### Récupération du graphique "barline"

In [ ]:
barline = qonto.connect(QONTO_USER_ID, QONTO_API_KEY).statement.barline(date_from=DATE_FROM,
                                                                        date_to=DATE_TO)
barline

### Récupération des opérations par typologie

In [ ]:
cash_summary = qonto.connect(QONTO_USER_ID, QONTO_API_KEY).statement.summary(date_from=DATE_FROM,
                                                                             date_to=DATE_TO)
cash_summary

### Récupération des dernières transactions

In [ ]:
df_last = qonto.connect(QONTO_USER_ID, QONTO_API_KEY).statement.transactions(date_from=LAST_TRANSACTIONS,
                                                                             date_to=DATE_TO)
df_last

## Model

### Calcule du solde courant

In [ ]:
current_position = round(df_statement['POSITION'].tolist()[-1], 2)
current_position

### Sauvegarde du relevé de compte et partage du fichier Excel

In [ ]:
df_statement.to_excel(TABLE_FILE)
statement_link = naas.asset.add(TABLE_FILE)

### Sauvegarde et partage du graphique en tant qu'image et page html

In [ ]:
# Image
barline.write_image(GRAPH_IMG)
graph_img = naas.asset.add(GRAPH_IMG)

# HTML
barline.write_html(GRAPH_FILE)
params = {"inline": True}
graph_link = naas.asset.add(GRAPH_FILE, params=params)

### Creation de l'email

In [ ]:
email_content = qonto.connect(QONTO_USER_ID, QONTO_API_KEY).statement.email(DATE_FROM,
                                                                            DATE_TO,
                                                                            current_position,
                                                                            graph_img,
                                                                            graph_link,
                                                                            cash_summary,
                                                                            LAST_TRANSACTIONS,
                                                                            df_last,
                                                                            statement_link)

## Output

### Envoi de l'email

In [ ]:
naas.notification.send(EMAIL_TO,
                       EMAIL_SUBJECT,
                       email_content)

### Automatiser l'envoi

In [ ]:
naas.scheduler.add(cron="0 8 * * 1")